In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

spaceship_titanic_path = kagglehub.competition_download('spaceship-titanic')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
titanic_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
titanic_df.CryoSleep.unique()


In [ ]:
df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')


In [ ]:
print(titanic_df.head())
print(titanic_df.dtypes)


In [ ]:
def preprocess(df):
    df = df.copy()

    # Drop Name (not useful)
    df = df.drop('Name', axis=1)


    # Split PassengerId into Group and GroupMember
    df[['Group', 'GroupMember']] = df['PassengerId'].str.split('_', expand=True)
    df['Group'] = df['Group'].astype(int)
    df['GroupMember'] = df['GroupMember'].astype(int)

    # Convert CryoSleep & VIP to binary
    df['CryoSleep'] = df['CryoSleep'].map({True: 1, False: 0})
    df['VIP'] = df['VIP'].map({True: 1, False: 0})

    df['TotalSpending'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['SpendPerAge'] = df['TotalSpending'] / (df['Age'] + 1)
    df['IsHighSpender'] = (df['TotalSpending'] > df['TotalSpending'].median()).astype(int)


    # Split Cabin into Deck / Num / Side
    df[['Deck', 'CabinNum', 'Side']] = df['Cabin'].str.split('/', expand=True)
    df.drop(columns=['Cabin'], inplace=True)
    df['CabinNum'] = pd.to_numeric(df['CabinNum'], errors='coerce')
    df['IsPortSide'] = (df['Side'] == 'P').astype(int)

    # Fill missing numerical with median
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())

    # Fill missing categorical with mode
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    # One-hot encode categorical features
    df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', 'Deck'], drop_first=True)
    df[['RoomService','FoodCourt','Spa','VRDeck','TotalSpending']] = np.log1p(df[['RoomService','FoodCourt','Spa','VRDeck','TotalSpending']])

    # Drop PassengerId
    df = df.drop('PassengerId', axis=1)
    df = df.drop('Side', axis=1)

    return df
titanic_df = preprocess(titanic_df)


In [ ]:
titanic_df.head(20)


In [ ]:
titanic_df.info()

In [ ]:
num_col = titanic_df.select_dtypes(['int64','float64'])
import seaborn as sns
import matplotlib.pyplot as plt
fig,ax = plt.subplots(3,5,figsize=(12,8))
ax = ax.flatten()

for i,col in enumerate(num_col):
    sns.boxplot(data=titanic_df, x=col, ax=ax[i], color='skyblue')
    ax[i].set_title(col)
plt.tight_layout()
plt.show()

In [ ]:
titanic_df.isna().sum()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = titanic_df.drop('Transported', axis=1)
y= titanic_df['Transported']
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3, random_state=42  )

In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

params = {
     'n_estimators': [200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.05, 0.1, 0.15,0.2],
    'gamma': [0, 0.1, 0.3, 0.5],
    'reg_alpha': [0, 0.01, 0.1, 0.5]

}
model = XGBClassifier(random_state=30)
rs = RandomizedSearchCV(model,param_distributions=params, n_iter=20, cv=5, scoring='accuracy')

rs.fit(X_train,y_train,eval_set=[(X_val, y_val)],  verbose=False)

print(rs.best_params_)
print("Best Score:", rs.best_score_)

In [ ]:
y_pred = rs.best_estimator_.predict(X_val)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Test Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
sns.heatmap(confusion_matrix(y_val, y_pred), annot=True, fmt='d')


In [ ]:

test_df = preprocess(df_test)

In [ ]:

y_pred1 = rs.predict(test_df)
y_pred1

In [ ]:
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Transported': y_pred1.astype(bool)
})


In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)
